## calculate the pO2 for each grid point at each depth for FOSI hindcast

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
var1 = 'O2' # mmol/m3
var2 = 'TEMP' # deg C
var3 = 'SALT' # PSS
var = [var1,var2,var3]

### DPLE hindcast

In [3]:
%%time
datadir = '/glade/campaign/cesm/collections/CESM1-DPLE/DPLE-FOSI/rerun_with_extension/timeseries'
dso = xr.Dataset()
#
for vv in var:
    filename = '%s/g.DPLE.GECOIAF.T62_g16.009.chey.pop.h.%s.024901-031812.nc' % (datadir,vv)
    ds_hind = xr.open_dataset(filename)
    dso[vv] = ds_hind[vv].compute()

CPU times: user 4min 50s, sys: 44 s, total: 5min 34s
Wall time: 5min 53s


In [4]:
for vv in ['TLONG', 'TLAT', 'z_t', 'time', 'time_bound', 'KMT', 'TAREA', 'dz']:
    dso[vv] = ds_hind[vv] 

### save T, S, O2

In [5]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dso.load()
dso.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc', mode='w')

### calculate pO2 based on O2, T, S, depth

In [6]:
depth = dso['z_t']*dso[var1]/dso[var1]/1e2 # m

In [7]:
dso['depth'] = depth.transpose("time","z_t","nlat","nlon")

In [8]:
dso['depth']

<xarray.DataArray 'depth' (time: 840, z_t: 60, nlat: 384, nlon: 320)>
array([[[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [ 4.9999995,  5.       ,  5.       , ...,        nan,
                 nan,        nan],
         ...,
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [15.       , 15.       , 15.       , ...,        nan,
                 nan,        nan],
...
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]]]], dtype=float32)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT     (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    TLONG    (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 318.9 319.4 319.8
    ULAT     (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    ULONG    (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 319.2 319.6 320.0
  * time     (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Dimensions without coordinates: nlat, nlon

In [9]:
%%time
pO2 = mi.compute_pO2(dso[var1],dso[var2],dso[var3],dso['depth'])

CPU times: user 27min 51s, sys: 23min 14s, total: 51min 6s
Wall time: 1h 32min 42s


In [10]:
pO2

<xarray.DataArray (time: 840, z_t: 60, nlat: 384, nlon: 320)>
array([[[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [0.30168462, 0.30192035, 0.30172515, ...,        nan,
                 nan,        nan],
         ...,
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [0.21501422, 0.21468447, 0.21446274, ...,        nan,
                 nan,        nan],
...
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]]]], dtype=float32)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT     (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    TLONG    (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 318.9 319.4 319.8
    ULAT     (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    ULONG    (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 319.2 319.6 320.0
  * time     (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Dimensions without coordinates: nlat, nlon

### save pO2

In [11]:
dsf = xr.Dataset()
dsf['pO2'] = pO2

In [12]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:  (z_t: 60, nlat: 384, nlon: 320, time: 840)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT     (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    TLONG    (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 318.9 319.4 319.8
    ULAT     (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    ULONG    (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 319.2 319.6 320.0
  * time     (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables:
    pO2      (time, z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan

In [13]:
%%time
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_pO2_1948-2017_monthly.nc', mode='w')

CPU times: user 98.1 ms, sys: 5.24 s, total: 5.34 s
Wall time: 6.64 s
